In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import dates as mdate
from datetime import datetime
import json, re, pymysql,  pandas
from pandas.io.json import json_normalize
#pandas.options.display.float_format = "{:.0f}".format
pandas.set_option('display.max_rows', None)
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
class BabiloniaPriceDataAnalytics:
    __CEAGESP=None
    
    def __init__(self, config_path):
        with open(config_path) as json_data_file:
            data = json.load(json_data_file)
            url = data['SQLALCHEMY_DATABASE_URI']
            self.__MUX_PORT_THRESHOLD = data['MUX_PORT_THRESHOLD']
        user, password, host, database = re.match('mysql://(.*?):(.*?)@(.*?)/(.*)', url).groups()
        self.__CONN = pymysql.connect(host=host,port=3306,user=user,passwd=password,db=database)
        
    def get_ceagesp_data(self):
        if self.__CEAGESP is None:
            query="""
                SELECT DATE, DATA
                FROM farmland.PRICES 
                WHERE SOURCE='CEAGESP';
                """
            df = pandas.read_sql_query(query, self.__CONN)
            pandas.to_datetime(df['DATE'])
            df.set_index('DATE', inplace=True)
            self.__CEAGESP = df
        return self.__CEAGESP
    
    def get_ceagesp_dates(self):
        self.get_ceagesp_data()
        return self.__CEAGESP.index

    def get_product_price(self, product, product_type):
        self.get_ceagesp_data()
        results =[]
        for index, row in self.__CEAGESP.iterrows():
            jd = json_normalize(json.loads(json.loads(row['DATA'])))
            price = float( jd[(jd['Produto'] == product)& (jd['Classificacao']==product_type)]['Comun'].values[0].replace(',', '.'))
            result={'DATE': index, 'PRICE': price}
            results.append(result)
        df = pandas.DataFrame(results) 
        return df

In [ ]:
bpda = BabiloniaPriceDataAnalytics('../../server/app/config.json')


In [ ]:
print(bpda.get_product_price('ALFACE ROMANA','-'))


In [ ]:
print(bpda.get_product_price('ESCAROLA','EXTRA'))